In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta

In [ ]:
df = pd.read_csv(r"C:\Users\Robyi\Documents\Data Science Dataset\page.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.head()

In [ ]:
print(df["group"].value_counts())

summary = df.groupby("group")["converted"].agg(["sum", "count"])
summary.columns = ["conversions", "total_users"]
summary["conversion_rate"] = summary["conversions"] / summary["total_users"]

print(summary)

In [ ]:
alpha_prior = 1
beta_prior = 1

alpha_A = alpha_prior + summary.loc["A", "conversions"]
beta_A = beta_prior + summary.loc["A", "total_users"] - summary.loc["A", "conversions"]

alpha_B = alpha_prior + summary.loc["B", "conversions"]
beta_B = beta_prior + summary.loc["B", "total_users"] - summary.loc["B", "conversions"]

samples_A = beta.rvs(alpha_A, beta_A, size=10000)
samples_B = beta.rvs(alpha_B, beta_B, size=10000)

prob_B_better = np.mean(samples_B > samples_A)

print(f"🔹 Probabilitas bahwa halaman B lebih baik dari A: {prob_B_better:.4f}")

In [ ]:
plt.figure(figsize=(10, 5))
sns.kdeplot(samples_A, label="Page A (Conversion Rate)", shade=True)
sns.kdeplot(samples_B, label="Page B (Conversion Rate)", shade=True)
plt.axvline(summary.loc["A", "conversion_rate"], color="blue", linestyle="--", label="Mean A")
plt.axvline(summary.loc["B", "conversion_rate"], color="red", linestyle="--", label="Mean B")
plt.xlabel("Conversion Rate")
plt.ylabel("Density")
plt.legend()
plt.title("Posterior Distribution of Conversion Rates (Bayesian A/B Test)")
plt.show()